# Score the Credit Risk model with randomly chosen 1000 records

Jukka Ruponen / IBM, 2019-05-04

Since this Notebook will score with 1000 randomly selected records from the sample data, you may re-run the notebook whenever you need to make more scoring requests. The requests should not be the same as with previous runs.

**Insert WLM credentials**

In [1]:
# WML credentials are need in order to make scoring requests to the scoring endpoint of the model.
# To get these credentials, go to your WML service details in IBM Cloud, select "Service credentials" tab
# and view one of the credentials to copy/paste values on below.

wml_credentials = {
  "apikey": "IG......................Ji",
  "instance_id": "ce......................e9",
  "password": "d0......................2e",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "84......................58"
}

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190505094921-0001
KERNEL_ID = ed630fb3-b9cf-426d-9c14-806bc2dfa46e


In [3]:
#@hidden_cell
# WML credentials are need in order to make scoring requests to the scoring endpoint of the model
# To get these credentials, look at your WML service details, "Service credentials" section and copy/paste here

wml_credentials = {
  "apikey": "IG2mbInGbSkw8xZOhuTrxLdtCaBbH17opw1t6uoZvKJi",
  "instance_id": "ce16a175-2a90-4725-b08a-ded2dd5fbee9",
  "password": "d056958e-04f9-4108-833a-77d27d8e4a2e",
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "8486ffe9-d6c0-451a-ab3c-763d04e2d258"
}

**Pick up the Scoring End-point from WLM deployment for your 'credit-risk-deployment' model**

In [4]:
# This is the "Scoring enpoint URL" value for your deployed model ('credit-risk-deployment') in our example.
# To get the value, open your "credit-risk" model in the Watson Studio project,
# click "Deployments" tab, click on the "credit-risk-deployment" and select "Implementation" tab.
# Copy the "Scoring End-point" value on below (between the quotes):
scoring_url = "<Scoring End-point>"

In [5]:
#@hidden_cell
scoring_url = "https://us-south.ml.cloud.ibm.com/v3/wml_instances/ce16a175-2a90-4725-b08a-ded2dd5fbee9/deployments/d42c3148-ff12-499c-804a-d31cb333e2d1/online"

## After you have set all the values above, you may run the whole Notebook

Optionally instal wget, if not done already

In [6]:
!pip install wget

pyspark 2.3.0 requires py4j==0.10.6, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Target directory /home/spark/shared/user-libs/python3/__pycache__ already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/wget-3.2.dist-info already exists. Specify --upgrade to force replacement.
Target directory /home/spark/shared/user-libs/python3/wget.py already exists. Specify --upgrade to force replacement.


**Load sample records to score the model with**

In [7]:
# Load sample records from github
import os, wget

filename='german_credit_feed.json'
if not os.path.isfile(filename):
    filename = wget.download('https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_feed.json',\
                             out=filename)
print(filename)

german_credit_feed.json


**Read the data from file**

In [8]:
import json

with open(filename, 'r') as scoring_file:
    scoring_data = json.load(scoring_file)
    

**Select 1000 randomly chosen records from the data**

In [9]:
import random

fields = scoring_data['fields']
values = []
for _ in range(1000):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}


**Prepare and execute the scoring request**

In [10]:
import urllib3, requests

In [11]:
# Define scoring request headers
headers = urllib3.util.make_headers(basic_auth='{username}:{password}'.format(username=wml_credentials['username'], password=wml_credentials['password']))
url = '{}/v3/identity/token'.format(wml_credentials['url'])
response = requests.get(url, headers=headers)
mltoken = json.loads(response.text).get('token')

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

In [12]:
# With the scoring_url got from the deployment, make a new scoring request
response_scoring = requests.post(scoring_url, json=payload_scoring, headers=header)
print("Scoring response:")
print(json.loads(response_scoring.text))

Scoring response:
{'fields': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [['less_0', 9, 'prior_payments_delayed', 'car_new', 1602, 'less_100', '4_to_7', 2, 'male', 'none', 2, 'unknown', 40, 'stores', 'own', 1, 'skilled', 1, 'none', 'yes', 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, [21, [0, 4, 7, 8, 13, 14, 15, 16, 17, 18, 19, 20], [1.0, 1.0, 3.0, 1.0, 9.0, 1602.0, 2.0, 2.

Done.